In [1]:
# Importing packages

import time
import datetime
import csv
import pandas as pd
import numpy as np

In [2]:
# Importing income_year

income_year = pd.read_csv("Bloomberg/income_year.csv", encoding='ISO-8859-1')
print(income_year.head())

                Company       Domicile      Ticker Type Any Values?  1995  \
0    (CA)SUPERIOR COURT            NaN         NaN  NaN         BAD   0.0   
1     (CO)SUPREME COURT  United States  313372L US    P         BAD   0.0   
2  (IDN)MINISTRY OF FIN      Indonesia    1133Z IJ    U         BAD   0.0   
3  (IDN)OFFICE OF THE P            NaN         NaN  NaN         BAD   0.0   
4  (ISR)OFFICE OF PRIME            NaN         NaN  NaN         BAD   0.0   

   1996  1997  1998  1999  ...   2007  2008  2009  2010  2011  2012  2013  \
0   0.0   0.0   0.0   0.0  ...    0.0   0.0   0.0   0.0   0.0   0.0   0.0   
1   0.0   0.0   0.0   0.0  ...    0.0   0.0   0.0   0.0   0.0   0.0   0.0   
2   0.0   0.0   0.0   0.0  ...    0.0   0.0   0.0   0.0   0.0   0.0   0.0   
3   0.0   0.0   0.0   0.0  ...    0.0   0.0   0.0   0.0   0.0   0.0   0.0   
4   0.0   0.0   0.0   0.0  ...    0.0   0.0   0.0   0.0   0.0   0.0   0.0   

   2014  2015  2016  
0   0.0   0.0   0.0  
1   0.0   0.0   0.0  
2   0.0 

In [3]:
# Importing Company-ID dictionary

dictionary = pd.read_csv("Merged/real_companies_dict.csv", encoding='ISO-8859-1')
dictionary.columns = ['No', 'Company', 'Company ID']
print(dictionary.head())

   No               Company  Company ID
0   0    (CA)SUPERIOR COURT           0
1   1     (CO)SUPREME COURT           1
2   2  (IDN)MINISTRY OF FIN           2
3   3  (IDN)OFFICE OF THE P           3
4   4  (ISR)OFFICE OF PRIME           4


In [4]:
# Substituting Company name with its ID

income_year = pd.merge(income_year, dictionary, how='outer')
del income_year['Company']
del income_year['No']
cols = list(income_year)
cols.insert(0, cols.pop(cols.index('Company ID')))
income_year = income_year.ix[:, cols]
print(income_year.head())

   Company ID       Domicile      Ticker Type Any Values?  1995  1996  1997  \
0           0            NaN         NaN  NaN         BAD   0.0   0.0   0.0   
1           1  United States  313372L US    P         BAD   0.0   0.0   0.0   
2           2      Indonesia    1133Z IJ    U         BAD   0.0   0.0   0.0   
3           3            NaN         NaN  NaN         BAD   0.0   0.0   0.0   
4           4            NaN         NaN  NaN         BAD   0.0   0.0   0.0   

   1998  1999  ...   2007  2008  2009  2010  2011  2012  2013  2014  2015  \
0   0.0   0.0  ...    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
1   0.0   0.0  ...    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
2   0.0   0.0  ...    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
3   0.0   0.0  ...    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
4   0.0   0.0  ...    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

   2016  
0   0.0  
1   0.0  
2   0.0  
3   0.0  
4   0.0  

[

In [5]:
income_year = income_year.values.tolist()
#print(income_year)

In [39]:
# Creating a list of all links with ONLY their characteristics

final = []
for i in range(0, len(income_year)):
    for j in range(i+1, len(income_year)):
#for i in range(0, 30):
#    for j in range(i+1, 30):
        common_dom = 0
        # Adding Common Domicile
        if str(income_year[i][1]) != 'nan' and str(income_year[j][1]) != 'nan':
            if income_year[i][1] == income_year[j][1]:
                common_dom = 1         
        final.append([income_year[i][0], income_year[j][0], income_year[i][3], income_year[j][3], common_dom])

In [41]:
# Removing duplicates

final = pd.DataFrame(final, columns = ["Company 1", "Company 2", "Company 1 Type", "Company 2 Type", "Common Domicile?"])
print('Before:', len(final))
final = final[~final.duplicated()]
print('After:', len(final))
print(final.head())

15420681
15420681
   Company 1  Company 2 Company 1 Type Company 2 Type  Common Domicile?
0          0          1            NaN              P                 0
1          0          2            NaN              U                 0
2          0          3            NaN            NaN                 0
3          0          4            NaN            NaN                 0
4          0          5            NaN            NaN                 0


In [42]:
# Exporting regression table with only basic information

final.to_csv('Regressions/regression_table_info.csv')

In [93]:
# Importing BACK the final regression table

final = pd.read_csv("Regressions/regression_table_info.csv", encoding='ISO-8859-1', index_col = 0)

In [94]:
print(len(final))
print(final.head())

15420681
   Company 1  Company 2 Company 1 Type Company 2 Type  Common Domicile?
0          0          1            NaN              P                 0
1          0          2            NaN              U                 0
2          0          3            NaN            NaN                 0
3          0          4            NaN            NaN                 0
4          0          5            NaN            NaN                 0


In [95]:
# Importing direct and hidden relationships and adding a column with ones

direct = pd.read_csv("Merged/direct_connections.csv", encoding='ISO-8859-1')
del direct['Unnamed: 0']
direct.columns = ['Company 1', 'Company 2']
direct['Direct Link?'] = [1] * len(direct)
print('Direct:')
print(direct.head())

print(' ')

hidden = pd.read_csv("Merged/hidden_connections.csv", encoding='ISO-8859-1')
del hidden['Unnamed: 0']
hidden.columns = ['Company 1', 'Company 2']
hidden['Hidden Link?'] = [1] * len(hidden)
print('Hidden:')
print(hidden.head())

Direct:
   Company 1  Company 2  Direct Link?
0       5025       1874             1
1       5025       1137             1
2       5025       4753             1
3       1874       5025             1
4       1874       1137             1
 
Hidden:
   Company 1  Company 2  Hidden Link?
0       4822       3465             1
1       4822       4353             1
2       4822       3577             1
3       1773       3465             1
4       1773       4353             1


In [96]:
# Adding DIRECT relationships to the regression table

print('Before:', len(final), len(direct))
final = pd.merge(final, direct, how='left', on=['Company 1', 'Company 2'])
print('After:', len(final))
print(final.head())
print('Sum:', final['Direct Link?'].sum())
final['Direct Link?'] = np.where(final['Direct Link?'].isnull(), 0, 1)
print('Sum:', final['Direct Link?'].sum())
print(final.head())

Before: 15420681 48676
After: 15420681
   Company 1  Company 2 Company 1 Type Company 2 Type  Common Domicile?  \
0          0          1            NaN              P                 0   
1          0          2            NaN              U                 0   
2          0          3            NaN            NaN                 0   
3          0          4            NaN            NaN                 0   
4          0          5            NaN            NaN                 0   

   Direct Link?  
0           NaN  
1           NaN  
2           NaN  
3           NaN  
4           NaN  
Sum: 24338.0
Sum: 24338
   Company 1  Company 2 Company 1 Type Company 2 Type  Common Domicile?  \
0          0          1            NaN              P                 0   
1          0          2            NaN              U                 0   
2          0          3            NaN            NaN                 0   
3          0          4            NaN            NaN                 0   
4  

In [97]:
# Adding HIDDEN relationships to the regression table

print('Before:', len(final), len(hidden))
final = pd.merge(final, hidden, how='left', on=['Company 1', 'Company 2'])
print('After:', len(final))
print(final.head())
print('Sum:', final['Hidden Link?'].sum())
final['Hidden Link?'] = np.where(final['Hidden Link?'].isnull(), 0, 1)
print('Sum:', final['Hidden Link?'].sum())
print(final.head())

Before: 15420681 19492
After: 15420681
   Company 1  Company 2 Company 1 Type Company 2 Type  Common Domicile?  \
0          0          1            NaN              P                 0   
1          0          2            NaN              U                 0   
2          0          3            NaN            NaN                 0   
3          0          4            NaN            NaN                 0   
4          0          5            NaN            NaN                 0   

   Direct Link?  Hidden Link?  
0             0           NaN  
1             0           NaN  
2             0           NaN  
3             0           NaN  
4             0           NaN  
Sum: 9744.0
Sum: 9744
   Company 1  Company 2 Company 1 Type Company 2 Type  Common Domicile?  \
0          0          1            NaN              P                 0   
1          0          2            NaN              U                 0   
2          0          3            NaN            NaN                 0

In [98]:
# Importing income_correlations_year

income_matrix = pd.read_csv("Bloomberg/income_correlations_year.csv", encoding='ISO-8859-1', index_col = 0)
print(income_matrix.head())

    0   1   2   3   4   5   6   7   8   9  ...   5544  5545  5546  5547  5548  \
0 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...    NaN   NaN   NaN   NaN   NaN   
1 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...    NaN   NaN   NaN   NaN   NaN   
2 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...    NaN   NaN   NaN   NaN   NaN   
3 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...    NaN   NaN   NaN   NaN   NaN   
4 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...    NaN   NaN   NaN   NaN   NaN   

   5549  5550  5551  5552  5553  
0   NaN   NaN   NaN   NaN   NaN  
1   NaN   NaN   NaN   NaN   NaN  
2   NaN   NaN   NaN   NaN   NaN  
3   NaN   NaN   NaN   NaN   NaN  
4   NaN   NaN   NaN   NaN   NaN  

[5 rows x 5554 columns]


In [99]:
print(len(final))
print((len(income_matrix)-1)*len(income_matrix)/2)
print(len(income_matrix))

15420681
15420681.0
5554


In [87]:
# Creating a list of year correlations and adding it to the regression table - VERY LONG!

income_matrix_list = []
for i in range(len(income_matrix)-1):
    for j in range(i+1, len(income_matrix)):
        income_matrix_list.append(income_matrix[str(i)].loc[j])
print(len(income_matrix_list))

1.0
0.364229386142
0.364229386142
15420681


In [100]:
final['Income Year Corrs'] = income_matrix_list
final.to_csv("Regressions/regression_table_python.csv")

In [101]:
# Importing BACK the final regression table

final = pd.read_csv("Regressions/regression_table_python.csv", encoding='ISO-8859-1', index_col = 0)
print(final.head())

//anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


   Company 1  Company 2 Company 1 Type Company 2 Type  Common Domicile?  \
0          0          1            NaN              P                 0   
1          0          2            NaN              U                 0   
2          0          3            NaN            NaN                 0   
3          0          4            NaN            NaN                 0   
4          0          5            NaN            NaN                 0   

   Direct Link?  Hidden Link?  Income Year Corrs  
0             0             0                NaN  
1             0             0                NaN  
2             0             0                NaN  
3             0             0                NaN  
4             0             0                NaN  


In [102]:
# Importing ebit_correlations_year

ebit_matrix = pd.read_csv("Bloomberg/ebit_correlations_year.csv", encoding='ISO-8859-1', index_col = 0)
print(ebit_matrix.head())

    0   1   2   3   4   5   6   7   8   9  ...   5544  5545  5546  5547  5548  \
0 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...    NaN   NaN   NaN   NaN   NaN   
1 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...    NaN   NaN   NaN   NaN   NaN   
2 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...    NaN   NaN   NaN   NaN   NaN   
3 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...    NaN   NaN   NaN   NaN   NaN   
4 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...    NaN   NaN   NaN   NaN   NaN   

   5549  5550  5551  5552  5553  
0   NaN   NaN   NaN   NaN   NaN  
1   NaN   NaN   NaN   NaN   NaN  
2   NaN   NaN   NaN   NaN   NaN  
3   NaN   NaN   NaN   NaN   NaN  
4   NaN   NaN   NaN   NaN   NaN  

[5 rows x 5554 columns]


In [103]:
print(len(final))
print((len(ebit_matrix)-1)*len(ebit_matrix)/2)
print(len(ebit_matrix))

15420681
15420681.0
5554


In [104]:
# Creating a list of year correlations and adding it to the regression table - VERY LONG!

ebit_matrix_list = []
for i in range(len(ebit_matrix)-1):
    for j in range(i+1, len(ebit_matrix)):
        ebit_matrix_list.append(ebit_matrix[str(i)].loc[j])
print(len(ebit_matrix_list))

15420681


In [105]:
final['EBIT Year Corrs'] = ebit_matrix_list
final.to_csv("Regressions/regression_table_python.csv")

In [106]:
# Importing BACK the final regression table

final = pd.read_csv("Regressions/regression_table_python.csv", encoding='ISO-8859-1', index_col = 0)
print(final.head())

//anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


   Company 1  Company 2 Company 1 Type Company 2 Type  Common Domicile?  \
0          0          1            NaN              P                 0   
1          0          2            NaN              U                 0   
2          0          3            NaN            NaN                 0   
3          0          4            NaN            NaN                 0   
4          0          5            NaN            NaN                 0   

   Direct Link?  Hidden Link?  Income Year Corrs  EBIT Year Corrs  
0             0             0                NaN              NaN  
1             0             0                NaN              NaN  
2             0             0                NaN              NaN  
3             0             0                NaN              NaN  
4             0             0                NaN              NaN  


In [107]:
# Importing ebitda_correlations_year

ebitda_matrix = pd.read_csv("Bloomberg/ebitda_correlations_year.csv", encoding='ISO-8859-1', index_col = 0)
print(ebitda_matrix.head())

    0   1   2   3   4   5   6   7   8   9  ...   5544  5545  5546  5547  5548  \
0 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...    NaN   NaN   NaN   NaN   NaN   
1 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...    NaN   NaN   NaN   NaN   NaN   
2 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...    NaN   NaN   NaN   NaN   NaN   
3 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...    NaN   NaN   NaN   NaN   NaN   
4 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...    NaN   NaN   NaN   NaN   NaN   

   5549  5550  5551  5552  5553  
0   NaN   NaN   NaN   NaN   NaN  
1   NaN   NaN   NaN   NaN   NaN  
2   NaN   NaN   NaN   NaN   NaN  
3   NaN   NaN   NaN   NaN   NaN  
4   NaN   NaN   NaN   NaN   NaN  

[5 rows x 5554 columns]


In [108]:
print(len(final))
print((len(ebitda_matrix)-1)*len(ebitda_matrix)/2)
print(len(ebitda_matrix))

15420681
15420681.0
5554


In [109]:
# Creating a list of year correlations and adding it to the regression table - VERY LONG!

ebitda_matrix_list = []
for i in range(len(ebitda_matrix)-1):
    for j in range(i+1, len(ebitda_matrix)):
        ebitda_matrix_list.append(ebitda_matrix[str(i)].loc[j])
print(len(ebitda_matrix_list))

15420681


In [110]:
final['EBITDA Year Corrs'] = ebitda_matrix_list
final.to_csv("Regressions/regression_table_python.csv")

In [111]:
# Importing BACK the final regression table

final = pd.read_csv("Regressions/regression_table_python.csv", encoding='ISO-8859-1', index_col = 0)
print(final.head())

//anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


   Company 1  Company 2 Company 1 Type Company 2 Type  Common Domicile?  \
0          0          1            NaN              P                 0   
1          0          2            NaN              U                 0   
2          0          3            NaN            NaN                 0   
3          0          4            NaN            NaN                 0   
4          0          5            NaN            NaN                 0   

   Direct Link?  Hidden Link?  Income Year Corrs  EBIT Year Corrs  \
0             0             0                NaN              NaN   
1             0             0                NaN              NaN   
2             0             0                NaN              NaN   
3             0             0                NaN              NaN   
4             0             0                NaN              NaN   

   EBITDA Year Corrs  
0                NaN  
1                NaN  
2                NaN  
3                NaN  
4                Na

In [113]:
# Importing rock_solid_truth and Companies-IDs dictionary

truth = pd.read_csv("Merged/rock_solid_truth.csv", encoding='ISO-8859-1')
truth = truth[['Cname Short', 'Position', 'Chairman?', 'Board Member?']]
truth.columns = ['Company Name', 'Position', 'Chairman?', 'Board Member?']
truth['Company Name'] = truth['Company Name'].str.upper()
truth['Board Member?'] = np.where(truth['Board Member?'] == 'Y', 1, truth['Board Member?'])
truth['Board Member?'] = np.where(truth['Board Member?'] == 'N', 0, truth['Board Member?'])
truth['Chairman?'] = np.where(truth['Chairman?'] == 'Y', 1, truth['Chairman?'])
truth['Chairman?'] = np.where(truth['Chairman?'] == 'Y', 0, truth['Chairman?'])
print(len(truth))
print(truth.head())

dictionary = pd.read_csv('Merged/real_companies_dict.csv', encoding='ISO-8859-1')
print(len(dictionary))
print(dictionary.head())

6240
           Company Name  Position Chairman? Board Member?
0  ASIA PAYMENT SYSTEMS  Former P       NaN             Y
1  THOMAS LEE & PARTNER       NaN         Y             N
2  FITTEC INTERNATIONAL       NaN       NaN             Y
3  CIG YANGTZE PORTS PL       NaN       NaN             Y
4  SPARKLE ROLL GROUP L       NaN       NaN             Y
5554
   Unnamed: 0          Company Name  Company ID
0           0    (CA)SUPERIOR COURT           0
1           1     (CO)SUPREME COURT           1
2           2  (IDN)MINISTRY OF FIN           2
3           3  (IDN)OFFICE OF THE P           3
4           4  (ISR)OFFICE OF PRIME           4


In [119]:
# Removing duplicates

truth = truth[~truth['Company Name'].duplicated()]
print(len(truth))

5554


In [121]:
# Merging IDs and Truth

truth = pd.merge(dictionary, truth, how = 'left')
del truth['Company Name']
del truth['Unnamed: 0']
print(len(truth))
print(truth.head())

5554
   Unnamed: 0          Company Name  Company ID           Position Chairman?  \
0           0    (CA)SUPERIOR COURT           0              Judge       NaN   
1           1     (CO)SUPREME COURT           1           Justice        NaN   
2           2  (IDN)MINISTRY OF FIN           2  Inspector General       NaN   
3           3  (IDN)OFFICE OF THE P           3    Special Advisor       NaN   
4           4  (ISR)OFFICE OF PRIME           4     Chief of Staff       NaN   

  Board Member?  
0           NaN  
1           NaN  
2           NaN  
3           NaN  
4           NaN  


In [125]:
# Creating separate tables with TRUTH for Company 1 & Company 2

truth_1 = truth.copy()
truth_1.columns = ['Company 1', 'Position 1', 'Chairman 1?', 'Board Member 1?']
print(truth_1.head())
truth_2 = truth.copy()
truth_2.columns = ['Company 2', 'Position 2', 'Chairman 2?', 'Board Member 2?']
print(truth_2.head())
print(final.head())

   Company 1         Position 1 Chairman 1? Board Member 1?
0          0              Judge         NaN             NaN
1          1           Justice          NaN             NaN
2          2  Inspector General         NaN             NaN
3          3    Special Advisor         NaN             NaN
4          4     Chief of Staff         NaN             NaN
   Company 2         Position 2 Chairman 2? Board Member 2?
0          0              Judge         NaN             NaN
1          1           Justice          NaN             NaN
2          2  Inspector General         NaN             NaN
3          3    Special Advisor         NaN             NaN
4          4     Chief of Staff         NaN             NaN
   Company 1  Company 2 Company 1 Type Company 2 Type  Common Domicile?  \
0          0          1            NaN              P                 0   
1          0          2            NaN              U                 0   
2          0          3            NaN            NaN  

In [127]:
# Adding Positions to the final table

final_new = pd.merge(final, truth_1, how = 'left')
final_new_new = pd.merge(final_new, truth_2, how = 'left')
print(len(final_new_new))
print(final_new_new.head())

15420681
   Company 1  Company 2 Company 1 Type Company 2 Type  Common Domicile?  \
0          0          1            NaN              P                 0   
1          0          2            NaN              U                 0   
2          0          3            NaN            NaN                 0   
3          0          4            NaN            NaN                 0   
4          0          5            NaN            NaN                 0   

   Direct Link?  Hidden Link?  Income Year Corrs  EBIT Year Corrs  \
0             0             0                NaN              NaN   
1             0             0                NaN              NaN   
2             0             0                NaN              NaN   
3             0             0                NaN              NaN   
4             0             0                NaN              NaN   

   EBITDA Year Corrs Position 1 Chairman 1? Board Member 1?  \
0                NaN      Judge         NaN             NaN   

In [128]:
final = final_new_new.copy()
final.to_csv("Regressions/regression_table_python.csv")

In [3]:
# Importing BACK the final regression table

final = pd.read_csv("Regressions/regression_table_python.csv", encoding='ISO-8859-1', index_col = 0)
print(final.head())

//anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (3,11,12,13,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


   Company 1  Company 2 Company 1 Type Company 2 Type  Common Domicile?  \
0          0          1            NaN              P                 0   
1          0          2            NaN              U                 0   
2          0          3            NaN            NaN                 0   
3          0          4            NaN            NaN                 0   
4          0          5            NaN            NaN                 0   

   Direct Link?  Hidden Link?  Income Year Corrs  EBIT Year Corrs  \
0             0             0                NaN              NaN   
1             0             0                NaN              NaN   
2             0             0                NaN              NaN   
3             0             0                NaN              NaN   
4             0             0                NaN              NaN   

   EBITDA Year Corrs Position 1 Chairman 1? Board Member 1?  \
0                NaN      Judge         NaN             NaN   
1       

In [4]:
# Importing income_correlations_quarter

income_matrix_quarter = pd.read_csv("Bloomberg/income_correlations_quarter.csv", encoding='ISO-8859-1', index_col = 0)
print(income_matrix_quarter.head())

    0   1   2   3   4   5   6   7   8   9  ...   5544  5545  5546  5547  5548  \
0 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...    NaN   NaN   NaN   NaN   NaN   
1 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...    NaN   NaN   NaN   NaN   NaN   
2 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...    NaN   NaN   NaN   NaN   NaN   
3 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...    NaN   NaN   NaN   NaN   NaN   
4 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...    NaN   NaN   NaN   NaN   NaN   

   5549  5550  5551  5552  5553  
0   NaN   NaN   NaN   NaN   NaN  
1   NaN   NaN   NaN   NaN   NaN  
2   NaN   NaN   NaN   NaN   NaN  
3   NaN   NaN   NaN   NaN   NaN  
4   NaN   NaN   NaN   NaN   NaN  

[5 rows x 5554 columns]


In [5]:
print(len(final))
print((len(income_matrix_quarter)-1)*len(income_matrix_quarter)/2)
print(len(income_matrix_quarter))

15420681
15420681.0
5554


In [6]:
# Creating a list of year correlations and adding it to the regression table - VERY LONG!

income_matrix_list = []
for i in range(len(income_matrix_quarter)-1):
    for j in range(i+1, len(income_matrix_quarter)):
        income_matrix_list.append(income_matrix_quarter[str(i)].loc[j])
print(len(income_matrix_quarter))

5554


In [9]:
final['Income Quarter Corrs'] = income_matrix_list
final.to_csv("Regressions/regression_table_python.csv")

In [10]:
# Importing BACK the final regression table

final = pd.read_csv("Regressions/regression_table_python.csv", encoding='ISO-8859-1', index_col = 0)
print(final.head())

//anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (3,11,12,13,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


   Company 1  Company 2 Company 1 Type Company 2 Type  Common Domicile?  \
0          0          1            NaN              P                 0   
1          0          2            NaN              U                 0   
2          0          3            NaN            NaN                 0   
3          0          4            NaN            NaN                 0   
4          0          5            NaN            NaN                 0   

   Direct Link?  Hidden Link?  Income Year Corrs  EBIT Year Corrs  \
0             0             0                NaN              NaN   
1             0             0                NaN              NaN   
2             0             0                NaN              NaN   
3             0             0                NaN              NaN   
4             0             0                NaN              NaN   

   EBITDA Year Corrs Position 1 Chairman 1? Board Member 1?  \
0                NaN      Judge         NaN             NaN   
1       

In [11]:
# Importing ebit_correlations_quarter

ebit_matrix_quarter = pd.read_csv("Bloomberg/ebit_correlations_quarter.csv", encoding='ISO-8859-1', index_col = 0)
print(ebit_matrix_quarter.head())

    0   1   2   3   4   5   6   7   8   9  ...   5544  5545  5546  5547  5548  \
0 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...    NaN   NaN   NaN   NaN   NaN   
1 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...    NaN   NaN   NaN   NaN   NaN   
2 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...    NaN   NaN   NaN   NaN   NaN   
3 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...    NaN   NaN   NaN   NaN   NaN   
4 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...    NaN   NaN   NaN   NaN   NaN   

   5549  5550  5551  5552  5553  
0   NaN   NaN   NaN   NaN   NaN  
1   NaN   NaN   NaN   NaN   NaN  
2   NaN   NaN   NaN   NaN   NaN  
3   NaN   NaN   NaN   NaN   NaN  
4   NaN   NaN   NaN   NaN   NaN  

[5 rows x 5554 columns]


In [12]:
print(len(final))
print((len(ebit_matrix_quarter)-1)*len(ebit_matrix_quarter)/2)
print(len(ebit_matrix_quarter))

15420681
15420681.0
5554


In [13]:
# Creating a list of year correlations and adding it to the regression table - VERY LONG!

ebit_matrix_list = []
for i in range(len(ebit_matrix_quarter)-1):
    for j in range(i+1, len(ebit_matrix_quarter)):
        ebit_matrix_list.append(ebit_matrix_quarter[str(i)].loc[j])
print(len(ebit_matrix_quarter))

5554


In [14]:
final['EBIT Quarter Corrs'] = ebit_matrix_list
final.to_csv("Regressions/regression_table_python.csv")

In [15]:
# Importing BACK the final regression table

final = pd.read_csv("Regressions/regression_table_python.csv", encoding='ISO-8859-1', index_col = 0)
print(final.head())

//anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (3,11,12,13,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


   Company 1  Company 2 Company 1 Type Company 2 Type  Common Domicile?  \
0          0          1            NaN              P                 0   
1          0          2            NaN              U                 0   
2          0          3            NaN            NaN                 0   
3          0          4            NaN            NaN                 0   
4          0          5            NaN            NaN                 0   

   Direct Link?  Hidden Link?  Income Year Corrs  EBIT Year Corrs  \
0             0             0                NaN              NaN   
1             0             0                NaN              NaN   
2             0             0                NaN              NaN   
3             0             0                NaN              NaN   
4             0             0                NaN              NaN   

   EBITDA Year Corrs Position 1 Chairman 1? Board Member 1?  \
0                NaN      Judge         NaN             NaN   
1       

In [16]:
# Importing ebitda_correlations_quarter

ebitda_matrix_quarter = pd.read_csv("Bloomberg/ebitda_correlations_quarter.csv", encoding='ISO-8859-1', index_col = 0)
print(ebitda_matrix_quarter.head())

    0   1   2   3   4   5   6   7   8   9  ...   5544  5545  5546  5547  5548  \
0 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...    NaN   NaN   NaN   NaN   NaN   
1 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...    NaN   NaN   NaN   NaN   NaN   
2 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...    NaN   NaN   NaN   NaN   NaN   
3 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...    NaN   NaN   NaN   NaN   NaN   
4 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...    NaN   NaN   NaN   NaN   NaN   

   5549  5550  5551  5552  5553  
0   NaN   NaN   NaN   NaN   NaN  
1   NaN   NaN   NaN   NaN   NaN  
2   NaN   NaN   NaN   NaN   NaN  
3   NaN   NaN   NaN   NaN   NaN  
4   NaN   NaN   NaN   NaN   NaN  

[5 rows x 5554 columns]


In [17]:
print(len(final))
print((len(ebitda_matrix_quarter)-1)*len(ebitda_matrix_quarter)/2)
print(len(ebitda_matrix_quarter))

15420681
15420681.0
5554


In [18]:
# Creating a list of year correlations and adding it to the regression table - VERY LONG!

ebitda_matrix_list = []
for i in range(len(ebitda_matrix_quarter)-1):
    for j in range(i+1, len(ebitda_matrix_quarter)):
        ebitda_matrix_list.append(ebitda_matrix_quarter[str(i)].loc[j])
print(len(ebitda_matrix_quarter))

5554


In [19]:
final['EBITDA Quarter Corrs'] = ebitda_matrix_list
final.to_csv("Regressions/regression_table_python.csv")

In [21]:
# Importing BACK the final regression table

final = pd.read_csv("Regressions/regression_table_python.csv", encoding='ISO-8859-1', index_col = 0)
print(final.head())

//anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (3,11,12,13,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


   Company 1  Company 2 Company 1 Type Company 2 Type  Common Domicile?  \
0          0          1            NaN              P                 0   
1          0          2            NaN              U                 0   
2          0          3            NaN            NaN                 0   
3          0          4            NaN            NaN                 0   
4          0          5            NaN            NaN                 0   

   Direct Link?  Hidden Link?  Income Year Corrs  EBIT Year Corrs  \
0             0             0                NaN              NaN   
1             0             0                NaN              NaN   
2             0             0                NaN              NaN   
3             0             0                NaN              NaN   
4             0             0                NaN              NaN   

   EBITDA Year Corrs Position 1 Chairman 1? Board Member 1?  \
0                NaN      Judge         NaN             NaN   
1       

In [22]:
# Importing price_correlations_year

price_matrix_year = pd.read_csv("Bloomberg/price_correlations_year.csv", encoding='ISO-8859-1', index_col = 0)
print(price_matrix_year.head())

    0   1   2   3   4   5   6   7   8   9  ...   5544  5545  5546  5547  5548  \
0 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...    NaN   NaN   NaN   NaN   NaN   
1 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...    NaN   NaN   NaN   NaN   NaN   
2 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...    NaN   NaN   NaN   NaN   NaN   
3 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...    NaN   NaN   NaN   NaN   NaN   
4 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...    NaN   NaN   NaN   NaN   NaN   

   5549  5550  5551  5552  5553  
0   NaN   NaN   NaN   NaN   NaN  
1   NaN   NaN   NaN   NaN   NaN  
2   NaN   NaN   NaN   NaN   NaN  
3   NaN   NaN   NaN   NaN   NaN  
4   NaN   NaN   NaN   NaN   NaN  

[5 rows x 5554 columns]


In [23]:
print(len(final))
print((len(price_matrix_year)-1)*len(price_matrix_year)/2)
print(len(price_matrix_year))

15420681
15420681.0
5554


In [24]:
# Creating a list of year correlations and adding it to the regression table - VERY LONG!

price_matrix_list = []
for i in range(len(price_matrix_year)-1):
    for j in range(i+1, len(price_matrix_year)):
        price_matrix_list.append(price_matrix_year[str(i)].loc[j])
print(len(price_matrix_year))

5554


In [25]:
final['Price Year Corrs'] = price_matrix_list
final.to_csv("Regressions/regression_table_python.csv")

In [26]:
# Importing BACK the final regression table

final = pd.read_csv("Regressions/regression_table_python.csv", encoding='ISO-8859-1', index_col = 0)
print(final.head())

//anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (3,11,12,13,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


   Company 1  Company 2 Company 1 Type Company 2 Type  Common Domicile?  \
0          0          1            NaN              P                 0   
1          0          2            NaN              U                 0   
2          0          3            NaN            NaN                 0   
3          0          4            NaN            NaN                 0   
4          0          5            NaN            NaN                 0   

   Direct Link?  Hidden Link?  Income Year Corrs  EBIT Year Corrs  \
0             0             0                NaN              NaN   
1             0             0                NaN              NaN   
2             0             0                NaN              NaN   
3             0             0                NaN              NaN   
4             0             0                NaN              NaN   

   EBITDA Year Corrs Position 1 Chairman 1? Board Member 1?  \
0                NaN      Judge         NaN             NaN   
1       

In [27]:
# Importing price_correlations_quarter

price_matrix_quarter = pd.read_csv("Bloomberg/price_correlations_quarter.csv", encoding='ISO-8859-1', index_col = 0)
print(price_matrix_quarter.head())

    0   1   2   3   4   5   6   7   8   9  ...   5544  5545  5546  5547  5548  \
0 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...    NaN   NaN   NaN   NaN   NaN   
1 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...    NaN   NaN   NaN   NaN   NaN   
2 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...    NaN   NaN   NaN   NaN   NaN   
3 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...    NaN   NaN   NaN   NaN   NaN   
4 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...    NaN   NaN   NaN   NaN   NaN   

   5549  5550  5551  5552  5553  
0   NaN   NaN   NaN   NaN   NaN  
1   NaN   NaN   NaN   NaN   NaN  
2   NaN   NaN   NaN   NaN   NaN  
3   NaN   NaN   NaN   NaN   NaN  
4   NaN   NaN   NaN   NaN   NaN  

[5 rows x 5554 columns]


In [28]:
print(len(final))
print((len(price_matrix_quarter)-1)*len(price_matrix_quarter)/2)
print(len(price_matrix_quarter))

15420681
15420681.0
5554


In [29]:
# Creating a list of year correlations and adding it to the regression table - VERY LONG!

price_matrix_list = []
for i in range(len(price_matrix_quarter)-1):
    for j in range(i+1, len(price_matrix_quarter)):
        price_matrix_list.append(price_matrix_quarter[str(i)].loc[j])
print(len(price_matrix_quarter))

5554


In [30]:
final['Price Quarter Corrs'] = price_matrix_list
final.to_csv("Regressions/regression_table_python.csv")